Представлен датасет центра приюта животных, и вашей задачей будет обучить модель таким образом, чтобы по определенным признакам была возможность максимально уверенно предсказать метки 'Adoption' и 'Transfer' (столбец “outcome_type”).

Здесь вы вольны делать что угодно. Я хочу видеть от вас:

Проверка наличия/обработка пропусков

Проверьте взаимосвязи между признаками

Попробуйте создать свои признаки

Удалите лишние

Обратите внимание на текстовые столбцы. Подумайте, что можно извлечь полезного оттуда

Использование профайлера вам поможет.

Не забывайте, что у вас есть PCA (Метод главных компонент). Он может пригодиться.

Вспомните о всем, что я говорил на предыдущих занятиях. Не все будет пригодится, но в жизни вам никто не будет говорить, что использовать :)

Хорошим классификатором для этой задачи будет "Случайный лес" (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

Понимать суть работы "леса" не обязательно на данном этапе, но качество предсказаний будет выше, чем с линейным классификатором. (если желаете, вот гайд https://adataanalyst.com/scikit-learn/linear-classification-method/)

Желаю успеха :)

In [3]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re
import datetime as dt
from functools import reduce

In [331]:
df = pd.read_csv('aac_shelter_outcomes.csv')
df.shape

(78256, 12)

In [332]:
df.head(10)

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown
5,4 months,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male
6,1 year,A693700,Other,Squirrel Mix,Tan,2013-12-13T00:00:00,2014-12-13T12:20:00,2014-12-13T12:20:00,NaN,Suffering,Euthanasia,Unknown
7,3 years,A692618,Dog,Chihuahua Shorthair Mix,Brown,2011-11-23T00:00:00,2014-12-08T15:55:00,2014-12-08T15:55:00,*Ella,Partner,Transfer,Spayed Female
8,1 month,A685067,Cat,Domestic Shorthair Mix,Blue Tabby/White,2014-06-16T00:00:00,2014-08-14T18:45:00,2014-08-14T18:45:00,Lucy,NaN,Adoption,Intact Female
9,3 months,A678580,Cat,Domestic Shorthair Mix,White/Black,2014-03-26T00:00:00,2014-06-29T17:45:00,2014-06-29T17:45:00,*Frida,Offsite,Adoption,Spayed Female


Смотрим, какие у нас в целом есть метки.

In [333]:
print(df.outcome_type.unique())

['Transfer' 'Adoption' 'Euthanasia' 'Return to Owner' 'Died' 'Disposal'
 'Relocate' 'Missing' nan 'Rto-Adopt']


Убираем все, что не относится к Adoption и Transfer, которые будем предсказывать.

In [334]:
df = df[df['outcome_type'].isin(['Transfer', 'Adoption'])]
df.shape

(56611, 12)

In [335]:
df.head(10)

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
5,4 months,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male
7,3 years,A692618,Dog,Chihuahua Shorthair Mix,Brown,2011-11-23T00:00:00,2014-12-08T15:55:00,2014-12-08T15:55:00,*Ella,Partner,Transfer,Spayed Female
8,1 month,A685067,Cat,Domestic Shorthair Mix,Blue Tabby/White,2014-06-16T00:00:00,2014-08-14T18:45:00,2014-08-14T18:45:00,Lucy,NaN,Adoption,Intact Female
9,3 months,A678580,Cat,Domestic Shorthair Mix,White/Black,2014-03-26T00:00:00,2014-06-29T17:45:00,2014-06-29T17:45:00,*Frida,Offsite,Adoption,Spayed Female
11,2 years,A673652,Dog,Papillon/Border Collie,Black/White,2012-02-28T00:00:00,2014-03-28T14:39:00,2014-03-28T14:39:00,Fancy,Partner,Transfer,Neutered Male
12,2 months,A677679,Dog,Chihuahua Shorthair/Pomeranian,Black,2014-03-07T00:00:00,2014-05-26T19:10:00,2014-05-26T19:10:00,Kash,Foster,Adoption,Neutered Male


Смотрим описание полученного датасета.

In [336]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56611 entries, 0 to 78255
Data columns (total 12 columns):
age_upon_outcome    56609 non-null object
animal_id           56611 non-null object
animal_type         56611 non-null object
breed               56611 non-null object
color               56611 non-null object
date_of_birth       56611 non-null object
datetime            56611 non-null object
monthyear           56611 non-null object
name                38660 non-null object
outcome_subtype     29425 non-null object
outcome_type        56611 non-null object
sex_upon_outcome    56611 non-null object
dtypes: object(12)
memory usage: 5.6+ MB


Удалим столбцы, не принципиальные для предсказания - name, animal_id, monthyear (дублирует datetime), outcome_subtype, age_upon_outcome (текстовое описание разницы между datetime и date_of_birth). В этих столбцах существуют пропуски и по больешй части они не имеют значения для предсказания, т.к. несут в себе идентификаторы строк или описательные данные вроде Имени, которые (по идее) не должны влиять на результат выбытия из приюта.

In [337]:
df.drop(['animal_id', 'monthyear', 'name', 'outcome_subtype', 'age_upon_outcome'], axis=1, inplace=True)

In [338]:
df.head(10)

,animal_type,breed,color,date_of_birth,datetime,outcome_type,sex_upon_outcome
0,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,Transfer,Intact Male
1,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,Transfer,Spayed Female
2,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,Adoption,Neutered Male
3,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,Transfer,Neutered Male
5,Dog,Leonberger Mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,Transfer,Intact Male
7,Dog,Chihuahua Shorthair Mix,Brown,2011-11-23T00:00:00,2014-12-08T15:55:00,Transfer,Spayed Female
8,Cat,Domestic Shorthair Mix,Blue Tabby/White,2014-06-16T00:00:00,2014-08-14T18:45:00,Adoption,Intact Female
9,Cat,Domestic Shorthair Mix,White/Black,2014-03-26T00:00:00,2014-06-29T17:45:00,Adoption,Spayed Female
11,Dog,Papillon/Border Collie,Black/White,2012-02-28T00:00:00,2014-03-28T14:39:00,Transfer,Neutered Male
12,Dog,Chihuahua Shorthair/Pomeranian,Black,2014-03-07T00:00:00,2014-05-26T19:10:00,Adoption,Neutered Male


Поле outcome_type ввиду наличия только 2х значений можно преобразовать в бинарный вид. Допустим, это будет столбец Adoptation. 1 - адоптация, 0 -трансфер.

In [339]:
df['Adoptation'] = df.outcome_type.str.contains('Adoption').astype(int)
df.drop(['outcome_type'], axis=1, inplace=True)
df.head(10)

,animal_type,breed,color,date_of_birth,datetime,sex_upon_outcome,Adoptation
0,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,Intact Male,0
1,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,Spayed Female,0
2,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,Neutered Male,1
3,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,Neutered Male,0
5,Dog,Leonberger Mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,Intact Male,0
7,Dog,Chihuahua Shorthair Mix,Brown,2011-11-23T00:00:00,2014-12-08T15:55:00,Spayed Female,0
8,Cat,Domestic Shorthair Mix,Blue Tabby/White,2014-06-16T00:00:00,2014-08-14T18:45:00,Intact Female,1
9,Cat,Domestic Shorthair Mix,White/Black,2014-03-26T00:00:00,2014-06-29T17:45:00,Spayed Female,1
11,Dog,Papillon/Border Collie,Black/White,2012-02-28T00:00:00,2014-03-28T14:39:00,Neutered Male,0
12,Dog,Chihuahua Shorthair/Pomeranian,Black,2014-03-07T00:00:00,2014-05-26T19:10:00,Neutered Male,1


Далее поработаем с полем Пола, в котором также содержатся данные о стерилизации\кастрации.

In [340]:
print(df.sex_upon_outcome.unique())

['Intact Male' 'Spayed Female' 'Neutered Male' 'Intact Female' 'Unknown']


In [341]:
df[df.sex_upon_outcome == "Unknown"].shape

(2761, 7)

Т.к. Unknown составляет всего порядка 5% от общей суммы Adopted и Transfer, то не будем слишком сильно углубляться в дробление еще и по этому признаку. Выделим отдельно Пол, отдельно удаление репродуктивных свойств питомца.

sex: 1 - девочка, 0 - мальчик или Unknown
intact: 1 - не тронутый, 0 - кастрат\стерильная или Unknown

In [342]:
df['intact'] = df.sex_upon_outcome.str.contains('Intact').astype(int)
df['sex'] = df.sex_upon_outcome.str.contains('Female').astype(int)
df.drop(['sex_upon_outcome'], axis=1, inplace=True)
df.head()

,animal_type,breed,color,date_of_birth,datetime,Adoptation,intact,sex
0,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,0,1,0
1,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,0,0,1
2,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,1,0,0
3,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,0,0,0
5,Dog,Leonberger Mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,0,1,0


Теперь разберемся с породой.

In [343]:
[ x for x in df.breed.unique()]

['Domestic Shorthair Mix',
 'Beagle Mix',
 'Pit Bull',
 'Miniature Schnauzer Mix',
 'Leonberger Mix',
 'Chihuahua Shorthair Mix',
 'Papillon/Border Collie',
 'Chihuahua Shorthair/Pomeranian',
 'Rat Terrier Mix',
 'Pit Bull Mix',
 'German Shepherd/Labrador Retriever',
 'Domestic Medium Hair Mix',
 'Mastiff Mix',
 'Plott Hound Mix',
 'Labrador Retriever Mix',
 'Siamese',
 'Miniature Pinscher Mix',
 'Russian Blue Mix',
 'Cardigan Welsh Corgi/English Setter',
 'Border Collie Mix',
 'Parson Russell Terrier Mix',
 'Miniature Poodle Mix',
 'Australian Shepherd/Australian Cattle Dog',
 'Basenji/Chihuahua Longhair',
 'Siberian Husky/German Shepherd',
 'German Shepherd/Boxer',
 'Labrador Retriever/German Shepherd',
 'German Shepherd/Chow Chow',
 'Pit Bull/Labrador Retriever',
 'Yorkshire Terrier Mix',
 'Beagle/Chihuahua Longhair',
 'Rat',
 'German Shepherd Mix',
 'Domestic Longhair Mix',
 'Dachshund Mix',
 'Hamster Mix',
 'Chihuahua Shorthair/Dachshund',
 'Flat Coat Retriever Mix',
 'Chihuahua L

Из списка следует, что в нашем дата-сете достаточно много помесей. Некоторые помечены Mix, другие написаны через /. На всякий случай сведем все в нижний регистр, т.к. у меня нет уверенности, что где-то не попадется информация в нижнем. Вdедем список атрибутов помеси.

In [344]:
is_mix = ['mix', '/']
df['breed'] = df.breed.str.lower()


In [345]:
df['is_mix'] = df.breed.str.contains('|'.join(is_mix)).astype(int)
df.head()

,animal_type,breed,color,date_of_birth,datetime,Adoptation,intact,sex,is_mix
0,Cat,domestic shorthair mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,0,1,0,1
1,Dog,beagle mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,0,0,1,1
2,Dog,pit bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,1,0,0,0
3,Dog,miniature schnauzer mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,0,0,0,1
5,Dog,leonberger mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,0,1,0,1


In [346]:
df.breed = df.breed.str.replace(f'mix', '').str.strip()
df.head()

,animal_type,breed,color,date_of_birth,datetime,Adoptation,intact,sex,is_mix
0,Cat,domestic shorthair,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,0,1,0,1
1,Dog,beagle,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,0,0,1,1
2,Dog,pit bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,1,0,0,0
3,Dog,miniature schnauzer,White,2005-06-02T00:00:00,2014-06-15T15:50:00,0,0,0,1
5,Dog,leonberger,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,0,1,0,1


Т.к. может быть указано несколько пород через слэш, то переведем  такой вид к спискам для упрощения one hot encoding.

In [347]:
df.breed = [kv.split('/') for kv in df.breed]

In [348]:
df.head()

,animal_type,breed,color,date_of_birth,datetime,Adoptation,intact,sex,is_mix
0,Cat,[domestic shorthair],Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,0,1,0,1
1,Dog,[beagle],White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,0,0,1,1
2,Dog,[pit bull],Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,1,0,0,0
3,Dog,[miniature schnauzer],White,2005-06-02T00:00:00,2014-06-15T15:50:00,0,0,0,1
5,Dog,[leonberger],Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,0,1,0,1


In [349]:
Далее выделим столбцы пород.

SyntaxError: invalid syntax (<ipython-input-349-7d82c39d90db>, line 1)

In [350]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('breed')), index=df.index, columns=mlb.classes_))
df.head()

,animal_type,color,date_of_birth,datetime,Adoptation,intact,sex,is_mix,abyssinian,affenpinscher,...,turtle,vizsla,weimaraner,welsh springer spaniel,welsh terrier,west highland,whippet,wire hair fox terrier,wirehaired pointing griffon,yorkshire terrier
0,Cat,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Dog,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Dog,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Dog,White,2005-06-02T00:00:00,2014-06-15T15:50:00,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Dog,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


То же самое проделаем с Видом Животного и цветом.

In [351]:
df.color = [kv.split('/') for kv in df.color]

In [352]:
df.head()

,animal_type,color,date_of_birth,datetime,Adoptation,intact,sex,is_mix,abyssinian,affenpinscher,...,turtle,vizsla,weimaraner,welsh springer spaniel,welsh terrier,west highland,whippet,wire hair fox terrier,wirehaired pointing griffon,yorkshire terrier
0,Cat,[Orange Tabby],2014-07-07T00:00:00,2014-07-22T16:04:00,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Dog,"[White, Brown]",2012-11-06T00:00:00,2013-11-07T11:47:00,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Dog,"[Blue, White]",2013-03-31T00:00:00,2014-06-03T14:20:00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Dog,[White],2005-06-02T00:00:00,2014-06-15T15:50:00,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Dog,"[Brown, White]",2013-06-03T00:00:00,2013-10-07T13:06:00,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [353]:
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('color')), index=df.index, columns=mlb.classes_))
df.head()

,animal_type,date_of_birth,datetime,Adoptation,intact,sex,is_mix,abyssinian,affenpinscher,afghan hound,...,Silver Lynx Point,Silver Tabby,Tan,Torbie,Tortie,Tortie Point,Tricolor,White,Yellow,Yellow Brindle
0,Cat,2014-07-07T00:00:00,2014-07-22T16:04:00,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Dog,2012-11-06T00:00:00,2013-11-07T11:47:00,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Dog,2013-03-31T00:00:00,2014-06-03T14:20:00,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Dog,2005-06-02T00:00:00,2014-06-15T15:50:00,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,Dog,2013-06-03T00:00:00,2013-10-07T13:06:00,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


И с видом животного.

In [354]:
df.animal_type = [[kv] for kv in df.animal_type]

In [355]:
df.head()

,animal_type,date_of_birth,datetime,Adoptation,intact,sex,is_mix,abyssinian,affenpinscher,afghan hound,...,Silver Lynx Point,Silver Tabby,Tan,Torbie,Tortie,Tortie Point,Tricolor,White,Yellow,Yellow Brindle
0,[Cat],2014-07-07T00:00:00,2014-07-22T16:04:00,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,[Dog],2012-11-06T00:00:00,2013-11-07T11:47:00,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,[Dog],2013-03-31T00:00:00,2014-06-03T14:20:00,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,[Dog],2005-06-02T00:00:00,2014-06-15T15:50:00,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,[Dog],2013-06-03T00:00:00,2013-10-07T13:06:00,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [356]:
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('animal_type')), index=df.index, columns=mlb.classes_))
df.head()

,date_of_birth,datetime,Adoptation,intact,sex,is_mix,abyssinian,affenpinscher,afghan hound,airedale terrier,...,Tortie Point,Tricolor,White,Yellow,Yellow Brindle,Bird,Cat,Dog,Livestock,Other
0,2014-07-07T00:00:00,2014-07-22T16:04:00,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2012-11-06T00:00:00,2013-11-07T11:47:00,0,0,1,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,2013-03-31T00:00:00,2014-06-03T14:20:00,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
3,2005-06-02T00:00:00,2014-06-15T15:50:00,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
5,2013-06-03T00:00:00,2013-10-07T13:06:00,0,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0


In [362]:
df.drop(['date_of_birth','datetime'], axis=1, inplace=True)

In [363]:
X = df.drop(['Adoptation'], axis=1)
Y = df['Adoptation']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [364]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((39627, 390), (16984, 390), (39627,), (16984,))

In [365]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [366]:
clf.score(X_test, Y_test)

0.7682524729156853

Вероятность правильного предсказания 77% =)